In [68]:
from dotenv import load_dotenv
import os
import json
import requests
import pandas as pd
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 \
import Features, EmotionOptions
from functools import reduce
from math import sqrt

In [69]:
load_dotenv()
THRESHOLD = 1.083274218344552
MATRIZ_EMOCIONES_PERFECTA = '{ "emotion": {"sadness" : 0.005441, "joy" : 0.997533, "fear" : 0.009288, "disgust" : 0.001843, "anger" : 0.002108}}'
MATRIZ_PRUEBA = '{ "emotion": {"sadness" : 0.523502, "joy" : 0.060197, "fear" : 0.034915, "disgust" : 0.124649, "anger" : 0.257796}}'
print(json.loads(MATRIZ_EMOCIONES_PERFECTAS))

{'emotion': {'sadness': 0.005441, 'joy': 0.997533, 'fear': 0.009288, 'disgust': 0.001843, 'anger': 0.002108}}


In [75]:
def preprocesar_imagen(url):
    response = requests.get('https://api.imagga.com/v2/tags?image_url=%s' % url,
                            auth=(os.environ.get('IMAGGA_API_KEY'), os.environ.get('IMAGGA_API_SECRET')))
    return response.json()

def extraer_emociones(texto):
    authenticator = IAMAuthenticator(os.environ.get('IBM_API_KEY'))
    natural_language_understanding = NaturalLanguageUnderstandingV1(version='2022-04-07',authenticator=authenticator)
    natural_language_understanding.set_service_url(os.environ.get('IBM_URL_SERVICE'))
    
    response = natural_language_understanding.analyze(text=texto, features=Features(emotion=EmotionOptions())).get_result()
    return json.dumps(response, indent=2)

def calcular_ed(json1,json2):
    data1 = json.loads(json1)
    data2 = json.loads(json2)

    x = list(data1['emotion'].values())
    y = list(data2['emotion'].values())
    print(x)

    if len(x) != len(y):
        raise ValueError("Las matrices deben tener la misma longitud")

    resultado = reduce(lambda a, b: a + b, [(yi - xi)**2 for xi, yi in zip(x, y)])
    return sqrt(resultado)

def clasificar(ed,threshold):
    return 'positivo' if ed < threshold else 'negativo'

In [76]:
def traducir_texto(text):
    url = "https://nlp-translation.p.rapidapi.com/v1/translate"

    headers ={
        "X-RapidAPI-Key": os.environ.get('NLP_API_KEY'),
        "X-RapidAPI-Host": os.environ.get('NLP_API_HOST')
    }
    querystring = {"text":text,"to":"en"}
    response = requests.get(url, headers=headers, params=querystring)

    return response.json()

In [77]:
def promedio(matriz1, matriz2):
    #Obtener las dimensiones de ambas matrices
    fil = len(matriz1)
    col = len(matriz1)
    #Creamos la matriz promedio
    matriz_promedio = [[0] * col for _ in range(fil)]    
    #Multiplicación de matrices
    for i in range(fil):
        for j in range (col):
            promedio = (matriz1[i][j] + matriz2[i][j]) / 2
            matriz_promedio[i][j] = promedio
    return matriz_promedio

In [79]:
print(preprocesar_imagen('https://pbs.twimg.com/media/FzBxSvfaIAAdjyf?format=jpg&name=4096x4096'))

{'result': {'tags': [{'confidence': 51.6775512695312, 'tag': {'en': 'hand'}}, {'confidence': 37.8711090087891, 'tag': {'en': 'technology'}}, {'confidence': 33.4202003479004, 'tag': {'en': 'business'}}, {'confidence': 33.1062202453613, 'tag': {'en': 'computer'}}, {'confidence': 32.4468803405762, 'tag': {'en': 'equipment'}}, {'confidence': 32.034065246582, 'tag': {'en': 'stereo'}}, {'confidence': 28.8875274658203, 'tag': {'en': 'device'}}, {'confidence': 25.4467754364014, 'tag': {'en': 'notebook'}}, {'confidence': 24.1064777374268, 'tag': {'en': 'office'}}, {'confidence': 23.8379878997803, 'tag': {'en': 'laptop'}}, {'confidence': 22.682991027832, 'tag': {'en': 'communication'}}, {'confidence': 21.9873161315918, 'tag': {'en': 'work'}}, {'confidence': 21.7257595062256, 'tag': {'en': 'hands'}}, {'confidence': 20.6586265563965, 'tag': {'en': 'keyboard'}}, {'confidence': 20.643533706665, 'tag': {'en': 'holding'}}, {'confidence': 20.257869720459, 'tag': {'en': 'person'}}, {'confidence': 20.090

In [80]:
print(extraer_emociones("@VirginAmerica and it's a really big bad thing about it"))

{
  "usage": {
    "text_units": 1,
    "text_characters": 55,
    "features": 1
  },
  "language": "en",
  "emotion": {
    "document": {
      "emotion": {
        "sadness": 0.523046,
        "joy": 0.100242,
        "fear": 0.276363,
        "disgust": 0.050727,
        "anger": 0.051678
      }
    }
  }
}


In [81]:
print(calcular_ed(MATRIZ_PRUEBA,MATRIZ_EMOCIONES_PERFECTA))

[0.523502, 0.060197, 0.034915, 0.124649, 0.257796]
1.1081969079211509


In [82]:
print(clasificar(1.1081969079,THRESHOLD))

negativo


In [83]:
print(traducir_texto('Tome una piña colada y acabe ebrio'))

{'status': 200, 'from': 'es', 'to': 'en', 'original_text': 'Tome una piña colada y acabe ebrio', 'translated_text': {'en': 'Take a pina colada and end up drunk'}, 'translated_characters': 34}


In [84]:
matriz1 = [[1, 2, 3],
           [4, 5, 6]]

matriz2 = [[7, 8, 9],
           [10, 11, 12]]

print(promedio(matriz1,matriz2))

[[4.0, 5.0], [7.0, 8.0]]
